# <center> Elementy numerycznej algebry liniowej </center>

Rozwiązywanie układów równań liniowych jest jednym z podstawowych problemów metod numerycznych. Układy równań liniowych występują w wielu dziedzinach nauki i inżynierii. Stosuje się też w uczeniu maszynowym np. podczas regresji z błędem średniokwadratowym. 


Istnieje kilka metod rozwiązywania układów równań. Na dzisiejszych zajęciach zajmiemy się:
* eliminacją Gaussa bez oraz z wyborem elementu głównego,
* metodami iteracyjnymi.

Problem rozwiązywania układu równań liniowych będzie nam towarzyszły do końca zajęć z tego przedmiotu.

## Normy i wskaźniki uwarunkowania

Wrażliwość układu (zmiana rozwiązania) na niewielkie zaburzenia wektora `b` zależy od macierzy `A` i ocenia się ja za pomocą tzw. współczynnika lub [wskaźnika uwarunkowania macierzy](https://pl.wikipedia.org/wiki/Wskaźnik_uwarunkowania) (ang. *condition number*). Im wyższa wartość tego wskaźnika. tym macierz jest gorzej uwarunkowana. Wskaźnik uwarunkowania to iloczyn normy macierzy z normą jej odwrotności.

$$cond(A)=|A|_{p}\cdot|A^{-1}|_{p}$$
gdzie *p* oznacza jedną z norm macierzy.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

***Zadanie 1.***

Porównaj normy 1,2, $\infty$ następujących macierzy:
* [Hilberta](https://pl.wikipedia.org/wiki/Macierz_Hilberta): o wymiarach 5x5 i 15x15
* [Vandermonde'a](https://pl.wikipedia.org/wiki/Macierz_Vandermonde’a): o wymiarach 5x5 i 15x15
* losowej o wartościach z przedziału [0,1]:  o wymiarach 5x5 i 15x15
* $P=\left[\begin{array}{cccc}4 & 1 & -1 & 0 \\ 1 & 3 & -1 & 0 \\ -1 & -1 & 5 & 2 \\ 0 & 0 & 2 & 4\end{array}\right]$

Czy wśród powyższych macierzy jest macierz [diagonalnie dominująca](https://pl.wikipedia.org/wiki/Macierz_przekątniowo_dominująca)?


*Wskazówka: Do wyznaczenia norm możesz wykorzystać funkcję `numpy.linalg.norm`*

In [ ]:
| Macierz               | ‖·‖₁       | ‖·‖₂       | ‖·‖∞    |
| --------------------- | ---------- | ---------- | ------- |
| **Hilbert 5×5**       | 2.2833     | 1.5671     | 2.2833  |
| **Hilbert 15×15**     | 3.3182     | 1.8459     | 3.3182  |
| **Vandermonde 5×5**   | 33.8828    | 23.0140    | 31.0    |
| **Vandermonde 15×15** | 39455.3614 | 23704.5881 | 32767.0 |
| **Losowa 5×5**        | 3.2475     | 2.3515     | 2.8119  |
| **Losowa 15×15**      | 9.5389     | 7.5860     | 9.1848  |
| **P**                 | 9.0        | 7.0861     | 9.0     |


***Zadanie 2.***

Oblicz wskaźniki uwarunkowania macierzy z poprzedniego zadania.

*Wskazówka: Możesz wykorzystać funkcję `numpy.linalg.cond`.*

In [ ]:
| Macierz               | Cond(A)     |
| --------------------- | ----------- |
| **Hilbert 5×5**       | 4.77 × 10⁵  |
| **Hilbert 15×15**     | 3.68 × 10¹⁷ |
| **Vandermonde 5×5**   | 4.08 × 10⁴  |
| **Vandermonde 15×15** | 2.96 × 10¹⁸ |
| **Losowa 5×5**        | 6.55 × 10³  |
| **Losowa 15×15**      | 8.54 × 10¹  |
| **P**                 | 3.54        |


## Rozwiązywanie układów równań metodą eliminacji Gaussa

***Zadanie 3.***

Jedną z metod rozwiązywania układów równań liniowych jest metoda eliminacji Gaussa. Metoda ta występuje w kilku odmianach. Poza podstawowym wariantem, możliwe jest zastosowanie metody z wyborem elementu głownego (tzw. *pivoting*). 

Celem tego zadania jest porównanie błędów rozwiązania otrzymanego z tych dwóch wariantów eliminacji Gaussa. Poniżej znajdują się implementacje obu tych metod. Każda z funkcji przyjmuje macierz `A` oraz wektor prawej strony równania `b`.

Samo polecenie znajduje się poniżej.

In [ ]:
def gauss_pivot(A, b):
    A=A.copy()
    b=b.copy()
    n = len(b)
    for k in range(n-1):
        ind_max = k
        for j in range(k+1, n):
            if abs(A[j,k]) > abs(A[ind_max,k]):
                ind_max = j
        if ind_max > k:
            tmp = A[ind_max,k:n].copy()
            A[ind_max,k:n] = A[k,k:n]
            A[k,k:n] = tmp
            tmp = b[ind_max].copy()
            b[ind_max] = b[k]
            b[k] = tmp
        akk = A[k,k]
        l = A[k+1:n,k] / akk
        for i in range(k+1, n):
            A[i,k] = 0
            A[i,k+1:n] = A[i,k+1:n] - l[i-k-1] * A[k,k+1:n]
            b[i] = b[i] - l[i-k-1] * b[k]
    x = np.zeros(n)
    x[n-1] = b[n-1]/A[n-1,n-1]
    for k in range(n-2, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:n], x[k+1:n])) / A[k,k]
    return x

In [ ]:
def gauss(A, b):
    A=A.copy()
    b=b.copy()
    n = len(b)
    for k in range(n-1):
        akk = A[k,k]
        l = A[k+1:n,k] / akk
        for i in range(k+1, n):
            A[i,k] = 0
            A[i,k+1:n] = A[i,k+1:n] - l[i-k-1] * A[k,k+1:n]
            b[i] = b[i] - l[i-k-1] * b[k]
    x = np.zeros(n)
    x[n-1] = b[n-1] / A[n-1,n-1]
    for k in range(n-2, -1, -1):
        x[k] = (b[k] - np.dot(A[k,k+1:n], x[k+1:n])) / A[k,k]
    return x

Stwórz macierze wartości losowych `A` o wymiarach 10x10 oraz wektor `b` o odpowiednich wymiarach. 
Chcemy rozwiązać układ równań `Ax=b` metodami eliminacji Gaussa bez oraz z wyborem elementu głównego, a następnie porównać dokładność wyników. Metoda z wyborem elementu głównego powinna dawać mniejszy błąd w przypadku dużych wartości znajdujących się na przekątnej. Sprawdź czy to prawda powtarzając obliczenia z  macierzami `A` zawierającym na pierwszym elemencie przekątnej coraz to mniejsze wartości (tak aby wzrosło znaczenie dalszych elementów na przękątnej i tym samym uaktywnił się wybór innego niż pierwszy elementu głównego).

Wskazówka:Do porównania możesz wykorzystać residuum. Jeżeli `x` jest rozwiązaniem układu to `Ax` powinno być równe `b`. Residuum to różnica pomiędzy `b` oraz `Ax`: `res=|b-Ax|`. Możesz porównać zawartości poszczególnych elementów lub obliczyć jakąś normę z otrzymanego wektora.

In [1]:
import numpy as np
import pandas as pd

def gauss_elimination(A, b):
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = len(b)
    for i in range(n):
        for j in range(i+1, n):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

def gauss_elimination_partial_pivoting(A, b):
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = len(b)
    for i in range(n):
        max_row = np.argmax(np.abs(A[i:, i])) + i
        if i != max_row:
            A[[i, max_row]] = A[[max_row, i]]
            b[[i, max_row]] = b[[max_row, i]]
        for j in range(i+1, n):
            factor = A[j, i] / A[i, i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]
    x = np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

# Parametry testu
n = 10
np.random.seed(42)
errors_no_pivot, errors_pivot = [], []
diagonal_values = 10.0 ** -np.arange(0, 10)

for val in diagonal_values:
    A = np.random.rand(n, n)
    A[0, 0] = val  # coraz mniejsze wartości na A[0,0]
    b = np.random.rand(n)

    x_np = gauss_elimination(A, b)
    x_pp = gauss_elimination_partial_pivoting(A, b)

    res_np = np.linalg.norm(b - A @ x_np)
    res_pp = np.linalg.norm(b - A @ x_pp)

    errors_no_pivot.append(res_np)
    errors_pivot.append(res_pp)

# Wyniki w tabeli
results_df = pd.DataFrame({
    'A[0,0]': diagonal_values,
    'Residuum (bez pivot)': errors_no_pivot,
    'Residuum (z pivotem)': errors_pivot
})

print(results_df)


         A[0,0]  Residuum (bez pivot)  Residuum (z pivotem)
0  1.000000e+00          2.348465e-15          8.641082e-16
1  1.000000e-01          8.547456e-15          5.661049e-16
2  1.000000e-02          5.271070e-14          1.310110e-15
3  1.000000e-03          3.831344e-13          8.527785e-16
4  1.000000e-04          3.737078e-12          8.025154e-16
5  1.000000e-05          1.317310e-11          7.049042e-16
6  1.000000e-06          7.315971e-10          7.484748e-15
7  1.000000e-07          1.166323e-09          6.004449e-16
8  1.000000e-08          1.922250e-08          1.355200e-15
9  1.000000e-09          1.864310e-07          9.485750e-16


## Metody iteracyjne

Innym sposobem na rozwiązanie układu równań liniowych jest wykorzystanie metod iteracyjnych, które generują ciągi przybliżeń wektora stanowiącego rozwiązanie układu. Państwa zadaniem będzie implementacja i porównanie zbieżności trzech najpopularniejszych metod iteracyjnego rozwiązywania układów równań liniowych

***Zadanie 4.***

Porównanie zbieżności metod Jacobiego, Gaussa-Seidla i Younga (SOR).
* Zaimplementuj solvery rozwiązujące układy równań metodami Jacobiego, Gaussa-Seidela  i Younga (SOR). Każda funkcja powinna przyjmować macierz A i wektor prawej strony b. Dla uproszczenia, dopuszczalne jest wykorzystanie  inv dla obliczenia macierzy odwrotnej do macierzy trójkątnej (w metodzie G-S i Younga).
* Porównaj zbieżność ciągów iteracyjnych otrzymanych 3 metodami dla 3 układów równań (3 macierzy). W metodzie Younga możesz przyjąć np. $ω = 1.2$.
* Dla macierzy, dla której metoda Younga okazała się zbieżna, porównaj zbieżność ciągów iteracyjnych otrzymanych dla wartości $0 < ω < 3$ (dodatkowe).
* Dla jakiej wartości parametru $ω$ zbieżność ciągu iteracyjnego jest najlepsza? Wynik otrzymany na podstawie obserwacji ciągu odchyleń od rozwiązania dokładnego należy porównać z wnioskiem płynącym z wykresu zależności promienia spektralnego macierzy iteracji w zależności od parametru $ω$ (dodatkowe).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tworzenie macierzy A i wektora b
np.random.seed(0)
n = 10
A = np.random.rand(n, n)
A = A + n * np.eye(n)  # Przekątniowo dominująca
x_true = np.ones(n)
b = A @ x_true

# Funkcja metody SOR
def sor_solver(A, b, omega, tol=1e-10, max_iter=10000):
    x = np.zeros_like(b)
    n = len(b)
    res = []
    for _ in range(max_iter):
        x_new = np.copy(x)
        for i in range(n):
            sigma1 = np.dot(A[i, :i], x_new[:i])
            sigma2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (1 - omega) * x[i] + omega * (b[i] - sigma1 - sigma2) / A[i, i]
        res_norm = np.linalg.norm(b - A @ x_new)
        res.append(res_norm)
        if res_norm < tol:
            break
        x = x_new
    return x, res

# Testujemy dla różnych wartości omega
omegas = np.linspace(0.1, 2.9, 30)
iterations = []

for omega in omegas:
    _, res = sor_solver(A, b, omega)
    iterations.append(len(res))

# Rysujemy wykres
plt.figure(figsize=(12, 6))
plt.plot(omegas, iterations, marker='o')
plt.xlabel('ω (parametr SOR)')
plt.ylabel('Liczba iteracji')
plt.title('Zbieżność metody SOR w zależności od ω')
plt.grid(True)
plt.show()


## Porównanie rozwiązania za pomocą metody `solve` oraz z użyciem odwrotności na przykładzie macierzy źle uwarunkowanej

***Zadanie 5.***

Dany jest układ równań $Hx=b$.
* H jest macierzą Hilberta o wymiarach $n=5x5$ (I przypadek) i $n=15x15$ (II przypadek),
* b jest wektorem o następujących elementach $b_i = 1/(n + i + 1)$ Uwaga: $i=1,\dots,n$.

Do rozwiązania układu wykorzystaj dwa algorytmy:
1. Z odwracaniem macierzy współczynników H.
2. Metodę `numpy.linalg.solve`.

Porównaj błędy obu rozwiązań. Aby ocenić błąd możesz:
* wyznaczyć wektor residuum otrzymanego rozwiązania,
* rozwiązać układ równań z innym wektorem $b$. Załóż, że wektor rozwiązania ma wszystkie elementy (współrzędne) równe 1 ($u_i = 1, i = 1, 2, . . . , n$). Wtedy $b = Hu$. Układ rozwiążemy bez korzystania z wiedzy o postaci $u$. Dopiero wynik porównamy ze znanym nam $u$.

In [4]:
import numpy as np

def hilbert_matrix(n):
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i, j] = 1 / (i + j + 1)
    return H

# Funkcja do tworzenia wektora b
def create_b(n):
    return np.array([1 / (n + i + 1) for i in range(n)])

# Funkcja do porównania metod
def compare_methods(H, b, u_true=None):
    # Odwracanie macierzy
    x_inv = np.linalg.inv(H) @ b
    res_inv = np.linalg.norm(H @ x_inv - b)

    # Solve z numpy
    x_solve = np.linalg.solve(H, b)
    res_solve = np.linalg.norm(H @ x_solve - b)

    # Porównanie z u, jeśli znane
    err_inv = np.linalg.norm(x_inv - u_true) if u_true is not None else None
    err_solve = np.linalg.norm(x_solve - u_true) if u_true is not None else None

    return {
        "Residuum (inv)": res_inv,
        "Residuum (solve)": res_solve,
        "Błąd względem u (inv)": err_inv,
        "Błąd względem u (solve)": err_solve
    }

# Przypadek 1: n = 5
n = 5
H = hilbert_matrix(n)  # tutaj poprawka
b = create_b(n)
u = np.ones(n)
b_known = H @ u

print("=== Przypadek n=5 ===")
print(compare_methods(H, b))
print(compare_methods(H, b_known, u_true=u))

# Przypadek 2: n = 15
n = 15
H = hilbert_matrix(n)  # tutaj poprawka
b = create_b(n)
u = np.ones(n)
b_known = H @ u

print("=== Przypadek n=15 ===")
print(compare_methods(H, b))
print(compare_methods(H, b_known, u_true=u))


=== Przypadek n=5 ===
{'Residuum (inv)': np.float64(6.252738611079423e-13), 'Residuum (solve)': np.float64(8.441528768080324e-17), 'Błąd względem u (inv)': None, 'Błąd względem u (solve)': None}
{'Residuum (inv)': np.float64(1.0845156636916551e-11), 'Residuum (solve)': np.float64(2.7194799110210365e-16), 'Błąd względem u (inv)': np.float64(9.159957913835852e-11), 'Błąd względem u (solve)': np.float64(3.7629505159417226e-12)}
=== Przypadek n=15 ===
{'Residuum (inv)': np.float64(0.2988578677362017), 'Residuum (solve)': np.float64(5.994417230841114e-16), 'Błąd względem u (inv)': None, 'Błąd względem u (solve)': None}
{'Residuum (inv)': np.float64(2.419377536588241), 'Residuum (solve)': np.float64(1.211110335867595e-15), 'Błąd względem u (inv)': np.float64(1738.3681104580382), 'Błąd względem u (solve)': np.float64(19.093927784534166)}


**Zadanie domowe. Znaczenie wskaźnika uwarunkowania macierzy w szacowaniu błędu rozwiązania**


Dana jest następująca macierz A współczynników układu dwóch równań liniowy:
$$A=\begin{bmatrix}10^5 & 9.9\cdot10^4\\1.00001& 0.99\end{bmatrix}$$

Wektor prawej strony równania $Ax=b$ dla rozwiązania x = $[1, 1]^T$ możemy wyznaczyć z równości $b = Ax$.

Należy:
* obliczyć wskaźnik uwarunkowania macierzy $A$,
* rozwiązać układ równań $Ax = b$ (nie korzystając z wiedzy o przyjętym rozwiązaniu dokładnym x) korzystając z funkcji `np.linalg.solve`,
* ocenić błąd otrzymanego rozwiązania i porównać go z błędem szacowanym za pomocą wskaźnika uwarunkowania macierzy A,
* przeprowadzić skalowanie tak, aby macierz $A$ była wyważona wierszami,
* wyznaczyć nowe wartości wektora b tak, aby rozwiązanie dokładne się nie
zmieniło,
* obliczyć wskaźnik uwarunkowania macierzy skalowanej,
* rozwiązać układ równań tą samą metodą jak poprzednio,
* ocenić błąd otrzymanego rozwiązania i porównać go z błędem szacowanym za pomocą wskaźnika uwarunkowania skalowanej macierzy $A$.
1. Czy błąd numeryczny rozwiązania w obu przypadkach jest tego samego rzędu?
2. Które szacowanie błędu jest bardziej zbliżone do faktycznego błędu?

In [5]:
import numpy as np

# Dane
A = np.array([[1e5, 9.9e4],
              [1.00001, 0.99]])
x_true = np.array([1, 1])

# Wyliczamy b = A x_true
b = A @ x_true

# 1. Wskaźnik uwarunkowania macierzy A (w normie 2)
cond_A = np.linalg.cond(A)

# 2. Rozwiązujemy Ax = b (bez wiedzy o x_true)
x_sol = np.linalg.solve(A, b)

# 3. Obliczamy błąd rzeczywisty i porównujemy z błędem szacowanym
error_real = np.linalg.norm(x_sol - x_true)
residuum = np.linalg.norm(b - A @ x_sol)

# Szacowany błąd: cond(A) * (relatywny błąd wektora b)
rel_error_b = np.linalg.norm(b - A @ x_true) / np.linalg.norm(b)  # tu jest zero, bo b = A x_true
# Ale w praktyce błąd powstaje w obliczeniach - możemy oszacować z residuum (b - A x_sol)
error_estimated = cond_A * residuum / np.linalg.norm(b)

print("Macierz A:")
print(A)
print(f"Wskaźnik uwarunkowania macierzy A: {cond_A:.2e}")
print(f"Rozwiązanie x_sol: {x_sol}")
print(f"Rzeczywisty błąd rozwiązania: {error_real:.2e}")
print(f"Residuum ||b - A x_sol||: {residuum:.2e}")
print(f"Szacowany błąd na podstawie wskaźnika uwarunkowania: {error_estimated:.2e}")

# 4. Skalowanie macierzy A (wyważenie wierszami)
row_norms = np.linalg.norm(A, axis=1)
D = np.diag(1 / row_norms)  # macierz diagonalna do skalowania wierszy
A_scaled = D @ A
b_scaled = D @ b

# 5. Wektor b po skalowaniu aby rozwiązanie się nie zmieniło (u_true to samo)
# Już jest b_scaled = D @ b = D @ A @ x_true = A_scaled @ x_true, więc jest OK

# 6. Wskaźnik uwarunkowania macierzy skalowanej
cond_A_scaled = np.linalg.cond(A_scaled)

# 7. Rozwiązujemy układ skalowany
x_sol_scaled = np.linalg.solve(A_scaled, b_scaled)

# 8. Oceniamy błąd i porównujemy z szacowaniem
error_real_scaled = np.linalg.norm(x_sol_scaled - x_true)
residuum_scaled = np.linalg.norm(b_scaled - A_scaled @ x_sol_scaled)
error_estimated_scaled = cond_A_scaled * residuum_scaled / np.linalg.norm(b_scaled)

print("\nPo skalowaniu wierszami:")
print("Macierz A_scaled:")
print(A_scaled)
print(f"Wskaźnik uwarunkowania macierzy A_scaled: {cond_A_scaled:.2e}")
print(f"Rozwiązanie x_sol_scaled: {x_sol_scaled}")
print(f"Rzeczywisty błąd rozwiązania skalowanego: {error_real_scaled:.2e}")
print(f"Residuum ||b_scaled - A_scaled x_sol_scaled||: {residuum_scaled:.2e}")
print(f"Szacowany błąd na podstawie wskaźnika uwarunkowania macierzy skalowanej: {error_estimated_scaled:.2e}")

# 9. Wnioski:
print("\nWnioski:")
print(f"1. Czy błąd numeryczny w obu przypadkach jest tego samego rzędu? {'Tak' if np.isclose(error_real, error_real_scaled, rtol=1e-1) else 'Nie'}")
print("2. Szacowanie błędu jest bardziej dokładne dla macierzy skalowanej." if error_estimated_scaled < error_estimated else
      "2. Szacowanie błędu jest bardziej dokładne dla macierzy oryginalnej.")


Macierz A:
[[1.00000e+05 9.90000e+04]
 [1.00001e+00 9.90000e-01]]
Wskaźnik uwarunkowania macierzy A: 2.00e+10
Rozwiązanie x_sol: [1. 1.]
Rzeczywisty błąd rozwiązania: 1.58e-11
Residuum ||b - A x_sol||: 2.22e-16
Szacowany błąd na podstawie wskaźnika uwarunkowania: 2.23e-11

Po skalowaniu wierszami:
Macierz A_scaled:
[[0.71065111 0.7035446 ]
 [0.71065463 0.70354104]]
Wskaźnik uwarunkowania macierzy A_scaled: 4.00e+05
Rozwiązanie x_sol_scaled: [1. 1.]
Rzeczywisty błąd rozwiązania skalowanego: 2.22e-11
Residuum ||b_scaled - A_scaled x_sol_scaled||: 0.00e+00
Szacowany błąd na podstawie wskaźnika uwarunkowania macierzy skalowanej: 0.00e+00

Wnioski:
1. Czy błąd numeryczny w obu przypadkach jest tego samego rzędu? Tak
2. Szacowanie błędu jest bardziej dokładne dla macierzy skalowanej.
